In [238]:
search_company = 'coromandel'

In [239]:
# company search
import requests
import json

headers = headers_dict = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
}

response = requests.request(
    "GET", 
    url = f"https://www.tijorifinance.com/api/v1/ind/company_search/?q={search_company}", 
    headers=headers
)

search_results = []
slug = ''

if not 200 <= response.status_code < 400:
    print(response.text)
else:
    for item in json.loads(response.text):
        if item['type'] == 'companies':
            search_results.append(
                {
                    'name' : item['name'],
                    'slug' : item['slug']
                }
            )
    if len(search_results) > 1:
        print(json.dumps(search_results, ensure_ascii=False, indent=4))
    elif len(search_results) == 1:
        slug = search_results[0]['slug']
        print("Single slug found and loaded, please skip next cell and proceed")
    else:
        print("No matching company found")

[
    {
        "name": "Coromandel Engineering Company Ltd.",
        "slug": "coromandel-engineering-company-limited"
    },
    {
        "name": "Coromandel International Ltd.",
        "slug": "coromandel-international-limited"
    }
]


In [240]:
slug = 'coromandel-international-limited'

In [241]:
# fetch particular company info

headers = headers_dict = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
}

response_overview = requests.request(
    "GET", 
    url = f"https://www.tijorifinance.com/company/{slug}", 
    headers=headers
)

if not 200 <= response_overview.status_code < 400:
    print(response_overview.text)
else:
    print(f"Success fetch overview - response_overview -> https://www.tijorifinance.com/company/{slug}")

response_financials = requests.request(
    "GET", 
    url = f"https://www.tijorifinance.com/company/{slug}/financials/", 
    headers=headers
)

if not 200 <= response_financials.status_code < 400:
    print(response_financials.text)
else:
    print(f"Success fetch financials - response_financials -> https://www.tijorifinance.com/company/{slug}/financials/")

response_benchmarking = requests.request(
    "GET", 
    url = f"https://www.tijorifinance.com/company/{slug}/benchmarking/", 
    headers=headers
)

if not 200 <= response_benchmarking.status_code < 400:
    print(response_benchmarking.text)
else:
    print(f"Success fetch benchamarking - response_benchmarking -> https://www.tijorifinance.com/company/{slug}/benchmarking/")
    
response_shareholding = requests.request(
    "GET", 
    url = f"https://www.tijorifinance.com/company/{slug}/shareholding/", 
    headers=headers
)

if not 200 <= response_shareholding.status_code < 400:
    print(response_shareholding.text)
else:
    print(f"Success fetch shareholding - response_shareholding -> https://www.tijorifinance.com/company/{slug}/shareholding/")

Success fetch overview - response_overview -> https://www.tijorifinance.com/company/coromandel-international-limited
Success fetch financials - response_financials -> https://www.tijorifinance.com/company/coromandel-international-limited/financials/
Success fetch benchamarking - response_benchmarking -> https://www.tijorifinance.com/company/coromandel-international-limited/benchmarking/
Success fetch shareholding - response_shareholding -> https://www.tijorifinance.com/company/coromandel-international-limited/shareholding/


In [242]:
# Get about, market cap, pe ratio

soup = BeautifulSoup(response_overview.text, 'html.parser')

data = {}
data['summary'] = soup.find('div', class_='about_desc detailed_desc').text.strip().split('\n')[0].strip()
data['market_cap'] = (int((soup.find('div', string='Market Cap').find_next('div', class_='company_details_value').text).replace('₹', '').replace(' Cr.', '').replace(',', '')) * 10**7 )
data['pe_ratio'] = float(soup.find('div', string='P/E').find_next('div', class_='company_details_value').text)


print(json.dumps(data, ensure_ascii=False, indent=4))

{
    "summary": "Coromandel International Limited is a leading agri-solutions provider in India, offering a wide range of products and services throughout the farming value chain. The company operates in segments such as Fertilizers, Specialty Nutrients, Crop Protection, and Retail. They are known for their Specialty Nutrient products, including Organic Fertilizers, and are a significant player in the Crop Protection business with a presence in India and global markets. Coromandel is a key manufacturer of Malathion and Phenthoate.",
    "market_cap": 479310000000,
    "pe_ratio": 32.85
}


In [243]:
# Get all FAQs list

from bs4 import BeautifulSoup
import json

faq_data = []

for html_code in [response_overview, response_financials, response_benchmarking, response_shareholding]:
    soup = BeautifulSoup(html_code.text, 'html.parser')
    
    faq_containers = soup.find_all('div', class_='faq_container')
    
    for container in faq_containers:
        question = container.find('div', class_='faq_data_header').find('span').text.strip()
        answer = container.find('div', class_='faq_answer').find('p').text.strip()
        
        faq_data.append(
            {
                'question': question,
                'answer': answer
            }
        )
    
print(json.dumps(faq_data, ensure_ascii=False, indent=4))

[
    {
        "question": "What does Coromandel Interntl. do?",
        "answer": "Coromandel International Limited is a leading agri-solutions provider in India, offering a wide range of products and services across the farming value chain including fertilizers, specialty nutrients, and crop protection."
    },
    {
        "question": "Who are the competitors of Coromandel Interntl.?",
        "answer": "Coromandel Interntl. major competitors are Chambal Fert & Chem, E.I.D. Parry (I), Fert Chem Travancore, Deepak Fert & Petro, RCF, GNFC, GSFC. Market Cap of Coromandel Interntl. is ₹50,886 Crs. While the median market cap of its peers are ₹14,076 Crs."
    },
    {
        "question": "Is Coromandel Interntl. financially stable compared to its competitors?",
        "answer": "Coromandel Interntl. seems to be financially stable compared to its competitors.The probability of it going bankrupt or facing a financial crunch seem to be lower than its immediate competitors."
    },
    {

In [244]:
# Get company forensics ( 1 - good, 2 - neutral, 3 - bad, 4 - not calculated )
soup = BeautifulSoup(response_overview.text, 'html.parser')

script_data = soup.find('script', id='company_details_data', type='application/json').text.strip()
forensics_data = json.loads(script_data)

if forensics_data['quick_look']:
    # Collect the factories
    forensics = forensics_data['quick_look']['data'][0]['factories']
    forensics.extend(forensics_data['quick_look']['data'][1]['factories'])
    
    forensics = sorted(forensics, key=lambda x: x['flag'], reverse=True)
    
    print(json.dumps(forensics, ensure_ascii=False, indent=4))
else:
    print('No forensics data available')

[
    {
        "name": "Revenue Recognition",
        "sentence": "Chances of aggressive revenue recognition - operating profit of company is not getting converted to cash and recievables rising faster than sales",
        "explanation": "A company might stuff the channel or book dummy sales. We check for quality of these sales and if profits are getting converted to cash by combing the P&L and Cash Flow statement.",
        "flag": 3
    },
    {
        "name": "Capex vs ROCE",
        "sentence": "Roce of the company has decreased even without significant capex",
        "explanation": "ROCE of companies in investment phase might be artificially depressed. We check for these and other reasons for ROCE trends of the company.\n\n",
        "flag": 3
    },
    {
        "name": "Current vs Historic Valuation",
        "sentence": "Company is trading at a premium to 3yr historic valuations",
        "explanation": "Current valuation metrics like PE, Price to book, EV by EBITDA etc are

In [245]:
# Get peers data as data frame

import pandas as pd
from IPython.display import HTML

def display_scrollable_dataframe(df_pivot):
    html = df_pivot.to_html(classes='table table-bordered', index=False)
    return HTML(f"""
    <div style="height:300px; overflow:auto; border:1px solid lightgrey; padding:5px;">
    {html}
    </div>
    """)

soup = BeautifulSoup(response_overview.text, 'html.parser')

script_data = soup.find('script', id='peers_table_data', type='application/json').text.strip()
peers_data = json.loads(script_data)

df_peers_data = pd.DataFrame(peers_data[1]['data'])

display_scrollable_dataframe(df_peers_data)

Latest Price,PE,Market Capital,Promoter,YoY Qtly Net Sales,ROCE (%),ROE (%),name,slug
1622.30,32.85,47931.0,57.31,-16.94,24.059216,17.433200,Coromandel Interntl.,coromandel-international-limited
497.95,14.4,19950.0,60.70,-11.74,19.765503,17.540266,Chambal Fert & Chem,chambal-fertilizers-chemicals-limited
811.00,16.31,14391.0,42.23,-3.98,20.557524,12.746361,E.I.D. Parry (I),eid-parry-india-limited
932.25,0.00,60323.0,90.00,-51.36,36.293217,10.624768,Fert Chem Travancore,fertilizers-and-chemicals-travancore-limited
1068.90,25.64,13539.0,45.63,-1.37,12.462464,8.182159,Deepak Fert & Petro,deepak-fertilizers-and-petrochemicals-corporation-limited
648.70,18.09,9532.0,41.30,22.34,6.919732,5.982361,GNFC,gujarat-narmada-valley-fertilizers-and-chemicals-limited
182.60,59.86,10074.0,75.00,8.73,7.390741,4.891288,RCF,rashtriya-chemicals-and-fertilizers-limited
214.00,15.82,8527.0,37.84,4.83,5.139833,4.431882,GSFC,gujarat-state-fertilizers-chemicals-limited


In [246]:
# Shareholding data as dataframe

soup = BeautifulSoup(response_shareholding.text, 'html.parser')

all_data = []

for script_tag in soup.find_all('script'):
    js_code = script_tag.string
    if js_code: 
        try:
            if 'var trendData' in js_code or 'var trendData2' in js_code:
                start_index = js_code.index('[') 
                end_index = js_code.rindex(']') + 1

                json_string = js_code[start_index:end_index]
                data = json.loads(json_string.replace("'", '"')) 

                all_data.append(data)
        except Exception as e:
            print(f"Error extracting JSON from script: {e}")

flattened_data = []
for dataset in all_data:
    for entry in dataset:
        name = entry['name']
        for item in entry['data']:
            timestamp = item[0]  # The timestamp
            value = item[1]      # The associated value
            flattened_data.append({'Name': name, 'Timestamp': timestamp, 'Value': value})



df = pd.DataFrame(flattened_data)
df['DateIST'] = pd.to_datetime(df['Timestamp'], unit='ms') + pd.Timedelta(hours=5, minutes=30)
df['DateIST'] = df['DateIST'].dt.strftime('%Y-%m')

df_shareholder_info = df.pivot_table(index='Name', columns='DateIST', values='Value')
df_shareholder_info.reset_index(inplace=True)

display_scrollable_dataframe(df_shareholder_info)

DateIST,Name,2008-03,2008-06,2008-09,2008-12,2009-03,2009-06,2009-09,2009-12,2010-03,2010-06,2010-09,2010-12,2011-03,2011-06,2011-09,2011-12,2012-03,2012-06,2012-09,2012-12,2013-03,2013-06,2013-09,2013-12,2014-03,2014-06,2014-09,2014-12,2015-03,2015-06,2015-09,2015-12,2016-03,2016-06,2016-09,2016-12,2017-03,2017-06,2017-09,2017-12,2018-03,2018-06,2018-09,2018-12,2019-03,2019-06,2019-09,2019-12,2020-03,2020-06,2020-09,2020-12,2021-03,2021-06,2021-09,2021-12,2022-03,2022-06,2022-09,2022-12,2023-03,2023-06,2023-09,2023-12,2024-03,2024-06
,DII-Others,0.05,0.05,0.05,0.05,0.05,0.05,0.06,0.05,0.05,0.05,0.05,0.05,0.04,0.12,0.08,0.08,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.06,0.05,0.04,0.05,0.04,0.05,0.04,0.83,0.65,0.55,0.39,0.17,0.16,0.15,0.15,0.21,0.39,0.64,0.62,0.60,0.60,0.61,0.65,0.68,0.58,0.60,0.58,0.60,0.53,0.33,0.28,0.25,0.40,0.47,0.46,0.45,0.46,0.36,0.52,0.63,0.78,0.93
,FII,5.16,1.96,2.59,2.68,2.85,2.73,3.18,3.26,3.55,3.82,4.14,5.28,5.56,6.17,6.69,7.71,7.73,7.63,7.28,7.27,6.49,6.27,6.57,7.50,6.80,7.00,6.65,6.55,6.74,6.67,5.45,8.26,4.62,4.82,4.45,4.53,5.16,5.29,5.64,5.85,5.58,4.36,3.76,3.59,4.13,5.89,5.03,5.18,5.14,5.90,7.50,8.68,9.37,8.96,9.28,8.85,8.72,9.67,11.57,11.90,11.04,9.20,8.82,9.34,8.97,9.15
,Individual < 2 lac,11.07,10.99,10.61,10.47,10.64,10.59,10.33,10.24,10.13,10.20,10.14,10.20,10.04,9.80,9.75,9.59,8.99,9.01,8.75,8.72,8.88,8.85,8.49,8.29,8.22,8.32,8.34,8.21,8.91,9.17,9.50,10.35,10.43,10.75,11.71,11.22,10.93,10.82,10.66,10.00,9.57,9.61,9.40,9.31,9.00,9.15,9.01,8.71,8.05,8.57,8.66,8.20,8.22,7.68,7.97,7.82,8.40,8.55,8.08,8.26,8.32,8.38,8.46,8.77,8.65,8.44
,Insurance Companies,4.69,4.69,4.47,4.46,4.25,3.71,3.68,3.67,3.60,3.19,3.13,2.83,2.63,2.48,2.31,2.28,2.18,2.18,2.18,2.19,2.31,2.32,2.33,1.74,1.65,1.57,1.43,1.34,1.30,1.32,1.39,1.39,1.34,1.28,1.27,1.19,1.17,1.12,1.11,0.97,0.89,0.90,0.90,0.86,0.76,0.76,0.75,0.50,0.41,0.37,0.26,0.19,1.52,2.56,0.26,0.26,0.26,0.18,3.34,2.40,2.58,2.89,3.24,3.10,4.13,4.57
,Mutual Funds,6.43,6.39,6.52,6.24,5.39,5.97,6.05,6.61,6.80,6.84,6.63,6.00,6.28,6.17,6.05,5.32,5.03,5.29,5.37,5.22,4.97,4.86,4.63,4.24,3.93,4.22,4.09,4.64,5.53,6.00,6.25,6.40,6.43,5.56,4.30,5.52,6.29,6.73,7.05,8.11,9.24,10.62,11.32,12.15,12.47,12.68,14.35,14.63,15.75,16.38,15.80,15.21,14.98,15.39,15.09,15.81,16.11,15.33,14.89,15.19,15.86,17.38,17.39,16.52,15.75,15.32
,Others,8.90,12.22,12.06,12.36,12.46,12.66,12.50,11.77,11.53,11.63,11.72,11.52,11.37,11.21,11.11,11.05,12.11,11.96,12.53,12.72,13.48,13.83,14.13,14.39,15.54,15.62,16.24,16.02,15.41,14.72,15.24,10.65,14.42,14.95,15.83,15.33,14.29,13.98,13.54,13.01,12.51,12.06,12.19,11.68,11.25,9.18,8.48,8.60,8.43,8.57,7.61,9.53,7.81,7.51,9.57,9.44,8.54,8.28,4.18,4.35,4.32,4.38,4.23,4.30,4.39,4.27
,Promoter,63.70,63.70,63.70,63.74,64.25,64.23,64.20,64.40,64.34,64.27,64.19,64.12,64.07,64.04,64.00,63.96,63.91,63.88,63.84,63.83,63.82,63.82,63.80,63.79,63.79,63.21,63.20,63.18,62.05,62.05,62.11,62.11,62.10,62.08,62.04,62.03,61.99,61.90,61.84,61.84,61.81,61.80,61.80,61.80,61.78,61.72,61.72,61.69,61.63,59.60,59.58,57.58,57.56,57.56,57.54,57.53,57.53,57.48,57.44,57.41,57.41,57.40,57.33,57.33,57.32,57.31
,Shares Pledged by Promoters(As a % of Total Promoter Holding),0.00,0.00,0.00,0.00,0.11,0.06,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.04,0.04,0.04,0.04,0.04,0.01,0.01,0.01,0.01,0.01,0.01
